## Interesting Visulizations
### Return the top-k interesting visulaizations from any database!
#### Submmitters: Ido Tausi, Afek Nahum

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [87]:

# Importing the dataset
dataset = pd.read_csv('StudentsDepression.csv')
target_column = 'Depression'

# dataset = pd.read_csv('BillionairesStatistics.csv')

dataset.head()


,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Male,33,Visakhapatnam,Student,5,0,8.97,2,0,5-6 hours,Healthy,B.Pharm,Yes,3,1.0,No,1
1,Female,24,Bangalore,Student,2,0,5.90,5,0,5-6 hours,Moderate,BSc,No,3,2.0,Yes,0
2,Male,31,Srinagar,Student,3,0,7.03,5,0,Less than 5 hours,Healthy,BA,No,9,1.0,Yes,0
3,Female,28,Varanasi,Student,3,0,5.59,2,0,7-8 hours,Moderate,BCA,Yes,4,5.0,Yes,1
4,Female,25,Jaipur,Student,4,0,8.13,3,0,5-6 hours,Moderate,M.Tech,Yes,1,1.0,No,0


In [88]:
def fill_missing_values(dataset):
    for column in dataset.columns:
        # print(f'{column} data type: {dataset[column].dtype}, is_numeric: {pd.api.types.is_numeric_dtype(dataset[column])}')
        if pd.api.types.is_numeric_dtype(dataset[column]):
            dataset.fillna(dataset[column].mean(), inplace=True)

    return dataset

fill_missing_values(dataset)


,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Male,33,Visakhapatnam,Student,5,0,8.97,2,0,5-6 hours,Healthy,B.Pharm,Yes,3,1.0,No,1
1,Female,24,Bangalore,Student,2,0,5.90,5,0,5-6 hours,Moderate,BSc,No,3,2.0,Yes,0
2,Male,31,Srinagar,Student,3,0,7.03,5,0,Less than 5 hours,Healthy,BA,No,9,1.0,Yes,0
3,Female,28,Varanasi,Student,3,0,5.59,2,0,7-8 hours,Moderate,BCA,Yes,4,5.0,Yes,1
4,Female,25,Jaipur,Student,4,0,8.13,3,0,5-6 hours,Moderate,M.Tech,Yes,1,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896,Female,27,Surat,Student,5,0,5.75,5,0,5-6 hours,Unhealthy,Class 12,Yes,7,1.0,Yes,0
27897,Male,27,Ludhiana,Student,2,0,9.40,3,0,Less than 5 hours,Healthy,MSc,No,0,3.0,Yes,0
27898,Male,31,Faridabad,Student,3,0,6.61,4,0,5-6 hours,Unhealthy,MD,No,12,2.0,No,0
27899,Female,18,Ludhiana,Student,5,0,6.88,2,0,Less than 5 hours,Healthy,Class 12,Yes,10,5.0,No,1


In [89]:
def bin_numerical_columns(dataframe):
    dataframe = dataframe.copy()
    for column in dataframe.columns:
        if pd.api.types.is_numeric_dtype(dataframe[column]):
            if dataframe[column].nunique() < 10:
                continue
            # remove outliers
            q1 = dataframe[column].quantile(0.25)
            q3 = dataframe[column].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            dataframe = dataframe[(dataframe[column] > lower_bound) & (dataframe[column] < upper_bound)]
            # TODO: decide on bins sizes
            bins_number = 10
            dataframe[column] = pd.cut(dataframe[column], bins=bins_number)
    return dataframe


In [93]:
def subset_data(dataframe, column, value):
    return dataframe[dataframe[column] == value]

def create_subsets_for_column(dataframe, column, subset_threshold=10):
    subsets = {}
    for value, count in dataframe[column].value_counts().items():
        print(f'{value}: {count}')
        if count < subset_threshold:
            continue
        subsets[value] = subset_data(dataframe, column, value)
    return subsets

def create_subsets(dataframe):
    subsets = {}
    for column in dataframe.columns:
        # TODO: calculate subset threshold
        subsets[column] = create_subsets_for_column(dataframe, column)
    return subsets

binned_dataset = bin_numerical_columns(dataset)
subsets_age = create_subsets_for_column(binned_dataset, 'City')
for key, value in subsets_age.items():
    print(f'{key}: {len(value)}')
    print(type(value))



# 
# subsets = {
#     "Age": {
#         (18, 20]: dataframe,
#         (20, 22]: dataframe,
#         ...
#     },
#     "CGPA": {
#         (6.5, 7.0]: dataframe,
#         (7.0, 7.5]: dataframe,
#         ...
#     },
#     ...
# }

Kalyan: 1570
Srinagar: 1371
Hyderabad: 1340
Vasai-Virar: 1290
Lucknow: 1153
Thane: 1139
Ludhiana: 1110
Agra: 1093
Surat: 1077
Kolkata: 1066
Jaipur: 1036
Patna: 1007
Visakhapatnam: 969
Pune: 967
Ahmedabad: 949
Bhopal: 933
Chennai: 883
Meerut: 824
Rajkot: 814
Delhi: 768
Bangalore: 767
Ghaziabad: 744
Mumbai: 699
Vadodara: 692
Varanasi: 685
Nagpur: 650
Indore: 643
Kanpur: 609
Nashik: 545
Faridabad: 461
Saanvi: 2
Bhavna: 2
City: 2
Harsha: 2
Kibara: 1
Nandini: 1
Nalini: 1
Mihir: 1
Nalyan: 1
M.Com: 1
ME: 1
Rashi: 1
Gaurav: 1
Reyansh: 1
Harsh: 1
Vaanya: 1
Mira: 1
Less than 5 Kalyan: 1
3: 1
Less Delhi: 1
M.Tech: 1
Khaziabad: 1
Kalyan: 1570
<class 'pandas.core.frame.DataFrame'>
Srinagar: 1371
<class 'pandas.core.frame.DataFrame'>
Hyderabad: 1340
<class 'pandas.core.frame.DataFrame'>
Vasai-Virar: 1290
<class 'pandas.core.frame.DataFrame'>
Lucknow: 1153
<class 'pandas.core.frame.DataFrame'>
Thane: 1139
<class 'pandas.core.frame.DataFrame'>
Ludhiana: 1110
<class 'pandas.core.frame.DataFrame'>
Agra: